# ABOUT:
- this notebook **trains LightGCN on three different BPR loss functions to explore if increasing negative item weights can improve GNN performance in CF task**
- Experiment details:
    - i devised two alternative BPR loss functions to the original BPR
    - steps:
        1. uniformly sample loss_neg_weight from the range (1, 2]
        2. perform a train test split on a unique seed
        3. train and evaluate LightGCN each of the three BPR functions
        4. Repeat steps 1-3 10 times

In [1]:
from recommenders.datasets.amazon_reviews import download_and_extract
from recommenders.datasets import movielens
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.utils.timer import Timer

import pandas as pd
from numpy.random import uniform, seed, randint

In [2]:
TOP_K = 10
MOVIELENS_DATA_SIZE = '100k'
EPOCHS = 50
BATCH_SIZE = 1024
SEED = DEFAULT_SEED  
COL_USER = "userID"
COL_ITEM = "itemID"

yaml_file = r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\code1\config\lightgcn.yml"
# user_file = "../../tests/resources/deeprec/lightgcn/user_embeddings.csv"
# item_file = "../../tests/resources/deeprec/lightgcn/item_embeddings.csv"

### read data

In [3]:
path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\FYP - GNN\data\ml-latest-small\ml-latest-small\ratings.csv"
df = pd.read_csv(path)
df.columns = ["userID", "itemID", "rating", "timestamp"] # better to follow the default colnames in the package

### Run Loss Function Study
- results are saved to wandb

In [4]:
n_trials = 10
loss_types = ["OriginalBPR", "AmpBPR1", "AmpBPR2"]

In [5]:
seed(1)
for _ in range(n_trials):
    
    # sample negative item weights - new sample every trial
    loss_neg_weight = uniform(1,2)
    
    # train test split - perform new split every trial 
    SEED0 = randint(0,100000)
    train, test = python_stratified_split(df, ratio=0.8,
                                          min_rating=1, filter_by='user', 
                                          col_user=COL_USER, col_item=COL_ITEM, 
                                          seed=SEED0)
    data = ImplicitCF(train = train, test=test, 
                      adj_dir=None, 
                      col_user=COL_USER, col_item=COL_ITEM, 
                      seed=SEED)
    
    # evaluate the loss_types
    for loss_type in loss_types: 
        # initiate parameters
        hparams = prepare_hparams(yaml_file,
                                  n_layers=3,
                                  loss_type = loss_type, 
                                  loss_neg_weight = loss_neg_weight, 
                                  batch_size=BATCH_SIZE,
                                  epochs=EPOCHS,
                                  learning_rate=0.01,
                                  eval_epoch=5,
                                  top_k=TOP_K,
                                 )
        # initiate model
        model = LightGCN(hparams, data, seed=SEED0)
        # train and evaluate
        with Timer() as train_time:
            model.fit()
#     print("Took {} seconds for training.".format(train_time.interval))

C:\Users\tanch\anaconda3\envs\fyp1\lib\site-packages\recommenders\models\deeprec\DataModel\ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)
wandb: Currently logged in as: chingfhen. Use `wandb login --relogin` to force relogin


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)6.0s: train loss = 0.36234 = (mf)0.36198 + (embed)0.00037
Epoch 2 (train)5.3s: train loss = 0.20207 = (mf)0.20132 + (embed)0.00075
Epoch 3 (train)5.0s: train loss = 0.17152 = (mf)0.17057 + (embed)0.00095
Epoch 4 (train)5.1s: train loss = 0.15649 = (mf)0.15534 + (embed)0.00115
Epoch 5 (train)5.0s + (eval)0.4s: train loss = 0.14465 = (mf)0.14330 + (embed)0.00135, recall = 0.10222, ndcg = 0.21258, precision = 0.18049, map = 0.05642
Epoch 6 (train)4.9s: train loss = 0.13715 = (mf)0.13561 + (embed)0.00154
Epoch 7 (train)4.9s: train loss = 0.12988 = (mf)0.12816 + (embed)0.00172
Epoch 8 (train)4.9s: train loss = 0.12030 = (mf)0.11838 + (embed)0.00192
Epoch 9 (train)4.9s: train loss = 0.11208 = (mf)0.10994 + (embed)0.00214
Epoch 10 (train)5.0s + (eval)0.3s: train loss = 0.10695 = (mf)0.10461 + (embed)0.00234, recall = 0.11840, ndcg = 0.23426, precision = 0.20115, map = 0.06733
Epoch

emb_loss,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_time,█▂▁▁▃▁▁▁▂▁
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
map,▁▃▅▅▇▇▇▇█▇
mf_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ndcg,▁▃▅▆▇█▇███
precision,▁▃▅▆▇█████
recall,▁▃▅▆▇▇▇███
train_time,█▃▁▂▁▁▁▁▂▄▃▁▂▆▃▅▃▃▄▃▃▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
emb_loss,0.00782


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)5.4s: train loss = 0.34464 = (mf)0.34433 + (embed)0.00030
Epoch 2 (train)5.0s: train loss = 0.20263 = (mf)0.20207 + (embed)0.00057
Epoch 3 (train)4.9s: train loss = 0.17270 = (mf)0.17195 + (embed)0.00075
Epoch 4 (train)4.9s: train loss = 0.14571 = (mf)0.14472 + (embed)0.00098
Epoch 5 (train)4.9s + (eval)0.3s: train loss = 0.12869 = (mf)0.12748 + (embed)0.00121, recall = 0.10719, ndcg = 0.22452, precision = 0.19262, map = 0.06171
Epoch 6 (train)5.0s: train loss = 0.11747 = (mf)0.11606 + (embed)0.00141
Epoch 7 (train)5.0s: train loss = 0.10976 = (mf)0.10818 + (embed)0.00158
Epoch 8 (train)4.9s: train loss = 0.10711 = (mf)0.10539 + (embed)0.00173
Epoch 9 (train)4.9s: train loss = 0.09938 = (mf)0.09749 + (embed)0.00189
Epoch 10 (train)4.9s + (eval)0.2s: train loss = 0.09273 = (mf)0.09069 + (embed)0.00205, recall = 0.11188, ndcg = 0.22899, precision = 0.20098, map = 0.06019
Epoch

emb_loss,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_time,█▂▁▁▂▁▂▂▅▇
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
map,▁▁▄▇▇▇████
mf_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ndcg,▁▁▄▇▇▇▇██▇
precision,▁▂▄▇▇▇▇██▇
recall,▁▂▄▇▇█████
train_time,█▃▂▂▃▂▂▁▁▁▂▂▂▂▂▁▁▁▁▁▁▂▂▂▂▂▂▂▇▆▂▂▁▆▃▆▃▃▄▅
emb_loss,0.00644


...

emb_loss,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_time,█▂▁▁▁▁▁▂▁▁
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
map,▁▃▂▅▆▆▇█▇▇
mf_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ndcg,▁▂▃▆▆▇▇█▇▇
precision,▁▂▃▆▇▇███▇
recall,▁▂▂▅▇▇▇██▇
train_time,▄▁▁▁▁█▄▃▃▅▂▁▁▁▁▁▂▂▂▃▂▃▁▃▁▂▁▂▂▁▁▂▁▁▂▁▂▁▂▂
emb_loss,0.00774


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)5.2s: train loss = 0.34020 = (mf)0.33991 + (embed)0.00029
Epoch 2 (train)4.9s: train loss = 0.19286 = (mf)0.19230 + (embed)0.00055
Epoch 3 (train)4.9s: train loss = 0.15823 = (mf)0.15748 + (embed)0.00075
Epoch 4 (train)4.9s: train loss = 0.14228 = (mf)0.14135 + (embed)0.00094
Epoch 5 (train)5.2s + (eval)0.5s: train loss = 0.12981 = (mf)0.12870 + (embed)0.00111, recall = 0.10827, ndcg = 0.23348, precision = 0.19934, map = 0.06150
Epoch 6 (train)5.2s: train loss = 0.11843 = (mf)0.11715 + (embed)0.00128
Epoch 7 (train)6.0s: train loss = 0.11254 = (mf)0.11111 + (embed)0.00144
Epoch 8 (train)6.2s: train loss = 0.10202 = (mf)0.10040 + (embed)0.00162
Epoch 9 (train)5.3s: train loss = 0.09410 = (mf)0.09230 + (embed)0.00180
Epoch 10 (train)5.0s + (eval)0.2s: train loss = 0.08644 = (mf)0.08446 + (embed)0.00198, recall = 0.11777, ndcg = 0.25442, precision = 0.21738, map = 0.06687
Epoch

emb_loss,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_time,█▁▁▁▁▁▁▁▁▁
loss,█▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
map,▁▃▅▇▇▇▇███
mf_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ndcg,▁▃▆███▇██▇
precision,▁▃▆███▇▇█▇
recall,▁▃▆▇▇▇▇▇██
train_time,▃▁▁▁▃▇█▄▂▁▁▂▁▂▂▂▁▁▃▂▂▁▁▂▂▂▂▁▂▁▁▂▂▂▁▁▂▁▂▁
emb_loss,0.00607


Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.
Epoch 1 (train)5.3s: train loss = 0.45070 = (mf)0.45040 + (embed)0.00030
Epoch 2 (train)5.0s: train loss = 0.27742 = (mf)0.27670 + (embed)0.00072
Epoch 3 (train)5.0s: train loss = 0.24617 = (mf)0.24523 + (embed)0.00094
Epoch 4 (train)5.3s: train loss = 0.22442 = (mf)0.22328 + (embed)0.00113
Epoch 5 (train)5.4s + (eval)0.5s: train loss = 0.20977 = (mf)0.20848 + (embed)0.00129, recall = 0.11737, ndcg = 0.24296, precision = 0.20984, map = 0.06578
Epoch 6 (train)5.2s: train loss = 0.19923 = (mf)0.19777 + (embed)0.00146
Epoch 7 (train)5.0s: train loss = 0.18755 = (mf)0.18592 + (embed)0.00163
Epoch 8 (train)5.0s: train loss = 0.17253 = (mf)0.17071 + (embed)0.00182
Epoch 9 (train)5.0s: train loss = 0.16336 = (mf)0.16134 + (embed)0.00202
Epoch 10 (train)5.0s + (eval)0.2s: train loss = 0.15664 = (mf)0.15443 + (embed)0.00221, recall = 0.13481, ndcg = 0.28050, precision = 0.23656, map = 0.07947
Epoch

emb_loss,▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_time,█▁▁▁▁▁▁▁▁▁
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
map,▁▆▇█▇██▇██
mf_loss,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
ndcg,▁▅▇█▇▇▇▇▇▇
precision,▁▅▇█▇▇▇▇▇▇
recall,▁▄▇███████
train_time,█▃▂█▅▃▂▂▁▂▄▂▂▂▂▂▂▂▁▁▂▁▁▃▂▂▂▁▂▃▂▂▂▂▂▂▂▂▁▂
emb_loss,0.00774
